In [1]:
# Import Taichi
import taichi as ti
import taichi.math as tm

[Taichi] version 1.7.0, llvm 15.0.1, commit 2fd24490, win, python 3.10.6


In [2]:
ti.init(arch=ti.gpu)

# type aliases
vec2 = tm.vec2
vec3 = tm.vec3

[Taichi] Starting on arch=cuda


In [3]:
# define resolution
WIDTH = 800
HEIGHT = 800
HALF_WIDTH = WIDTH / 2
HALF_HEIGHT = HEIGHT / 2

#define shape
PARTICLE_COUNT = 50

# define variables 
masses = ti.field(dtype=float, shape=PARTICLE_COUNT)
positions = ti.Vector.field(2, dtype=float, shape=PARTICLE_COUNT)
velocities = ti.Vector.field(2, dtype=float, shape=PARTICLE_COUNT)

@ti.func
def generate_random_float(minInclude, maxInclude):
    random_float = minInclude + (maxInclude - minInclude) * ti.random()
    return random_float

@ti.func
def random_position():
    return ti.Vector([generate_random_float(-HALF_WIDTH, HALF_WIDTH), generate_random_float(-HALF_HEIGHT, HALF_HEIGHT)]) / ti.Vector([WIDTH, HEIGHT]) + 0.5

@ti.kernel
def init():
    for i in range(PARTICLE_COUNT):
        positions[i] = random_position()

In [4]:
@ti.kernel
def update():
    pass

In [6]:
init()

gui = ti.GUI("Mass Spring System", res=(WIDTH, HEIGHT))
while gui.running:
    for i in range(PARTICLE_COUNT):
        gui.circle(pos=positions[i], radius=5, color=0xED553B)
    gui.circle(pos=vec2([20, 20]), radius=10, color=0xFFFFFF)
    gui.show()